In [ ]:
import os
from os import path

# Third-party
import astropy.coordinates as coord
from astropy.constants import G
from astropy.io import fits, ascii
from astropy.stats import median_absolute_deviation
from astropy.table import Table, QTable, join
from astropy.time import Time
import astropy.units as u
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
import numpy as np
%matplotlib inline
import h5py
import pandas as pd
from sqlalchemy import func
import tqdm
from scipy.stats import beta

from thejoker import JokerSamples

from twoface.config import TWOFACE_CACHE_PATH
from twoface.samples_analysis import MAP_sample
from twoface.db import (db_connect, AllStar, AllVisit, AllVisitToAllStar, NessRG,
                        StarResult, Status, JokerRun)
from twoface.plot import plot_two_panel, plot_phase_fold
from twoface.mass import get_m2_min, mf, period_at_surface, asini
from twoface.log import log as logger
logger.setLevel(100)

In [ ]:
plot_path = '../paper/1-catalog/figures/'
table_path = '../paper/1-catalog/tables/'

In [ ]:
Session, _ = db_connect(path.join(TWOFACE_CACHE_PATH, 'apogee.sqlite'))
session = Session()

samples_file = path.join(TWOFACE_CACHE_PATH, 'apogee-jitter.hdf5')
mcmc_samples_file = path.join(TWOFACE_CACHE_PATH, 'apogee-jitter-mcmc.hdf5')

In [ ]:
unimodal = QTable.read(path.join(table_path, 'highK-unimodal.fits'), 
                       astropy_native=True, character_as_bytes=False)
bimodal = QTable.read(path.join(table_path, 'highK-bimodal.fits'), 
                      astropy_native=True, character_as_bytes=False)

clean_uni = unimodal[(unimodal['clean_flag'] == 0)]
clean_conv_uni = clean_uni[clean_uni['converged']]
clean_bi = bimodal[(bimodal['clean_flag'] == 0)]# & (bimodal['LOGG'] > 0) & 
#                    (bimodal['TEFF'] > 0)]

print('{0} unimodal, {1} clean'.format(len(unimodal), len(clean_uni)))
print('{0} bimodal, {1} clean'.format(len(bimodal)//2, len(clean_bi)//2))

In [ ]:
cache_path = '../cache/yst'
os.makedirs(cache_path, exist_ok=True)

In [ ]:
tbl = clean_conv_uni[clean_conv_uni['LOGG'] > 2][clean_conv_uni.colnames[:17]]
tbl.write(path.join(cache_path, 'orbit-elements.csv'), format='ascii.ecsv')

In [ ]:
allvisits = fits.getdata('/Users/adrian/data/APOGEE_DR14/allVisit-l31c.2.fits')

In [ ]:
visits = allvisits[np.isin(allvisits['APOGEE_ID'], tbl['APOGEE_ID'])]

In [ ]:
len(visits)

In [ ]:
ndata = 0
for row in tbl:
    star = AllStar.get_apogee_id(session, row['APOGEE_ID'])
    data = star.apogeervdata()
    ndata += len(data)
ndata

In [ ]:
skip_mask = np.sum(2 ** np.array([3, 4])) # VERY_BRIGHT_NEIGHBOR, LOW_SNR
clean_visits = visits[np.isfinite(visits['VHELIO']) &
                            np.isfinite(visits['VRELERR']) &
                            (visits['VRELERR'] < 100.) & # MAGIC
                            ((visits['STARFLAG'] & skip_mask) == 0)]

In [ ]:
fits.BinTableHDU(clean_visits).writeto(path.join(cache_path, 'visits.fits'))